In [1]:
from __future__ import absolute_import
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import sklearn
from datetime import datetime

In [2]:
trn_df = pd.read_csv('./train/train.csv', encoding='utf8')
tst_df = pd.read_csv('./test/test.csv', encoding='utf8')
trn_df.head()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,113261,38038,513.80,5,0,N,0,N,N,...,0,33,172652.0,457,59333,N,0,102,0,516056
1,0,134508,45725,465.62,5,0,N,2,N,N,...,0,9,105114.0,451,0,N,5817,102,0,4376
2,6881,15408,188328,513.80,5,0,N,0,N,N,...,0,6,152458.0,457,59333,N,0,102,0,483434
3,6716,157159,29967,1016.11,5,62,N,5,N,N,...,0,5,172946.0,247,50436,N,3281,102,0,1407164
4,5975,105985,81305,713.66,5,62,N,4,N,N,...,0,6,182129.0,263,93775,N,5817,102,0,1051004


In [3]:
trn_df = trn_df.dropna(subset=['flbmk', 'flg_3dsmk'])

tst_df = tst_df.dropna(subset=['flbmk', 'flg_3dsmk'])

In [4]:
trn_df['loctm'] = trn_df.loctm.astype(int).astype(str).str.zfill(6)
trn_df['loctm'] = pd.to_datetime(trn_df.loctm, format='%H%M%S', errors='raise')
trn_df['loctm'] = trn_df.loctm.dt.time

tst_df['loctm'] = tst_df.loctm.astype(int).astype(str).str.zfill(6)
tst_df['loctm'] = pd.to_datetime(tst_df.loctm, format='%H%M%S', errors='raise')
tst_df['loctm'] = tst_df.loctm.dt.time

In [5]:
feature_list = [
    'locdt', 'loctm', 'contp', 'etymd', 'mcc', 'conam', 'ecfg', 'insfg', 'iterm', 'stocn', 'stscd', 'hcefg', 'csmcu', 'flbmk', 'flg_3dsmk'
]
trn_df = trn_df[feature_list + ['fraud_ind']]
tst_df = tst_df[feature_list]
trn_df.head()

,locdt,loctm,contp,etymd,mcc,conam,ecfg,insfg,iterm,stocn,stscd,hcefg,csmcu,flbmk,flg_3dsmk,fraud_ind
0,33,17:26:52,5,0,457,513.80,N,N,0,102,0,5,0,N,N,0
1,9,10:51:14,5,2,451,465.62,N,N,0,102,0,0,0,N,N,0
2,6,15:24:58,5,0,457,513.80,N,N,0,102,0,5,0,N,N,0
3,5,17:29:46,5,5,247,1016.11,N,N,0,102,0,5,62,N,N,0
4,6,18:21:29,5,4,263,713.66,N,N,0,102,0,5,62,N,N,0


In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import sys
import tensorflow as tf

# Method 2

In [7]:
target_encode_feature_list = ['contp', 'etymd', 'iterm', 'stocn', 'hcefg', 'csmcu']
one_hot_encode_feature_list = ['ecfg', 'insfg', 'flbmk', 'flg_3dsmk']

In [8]:
trn_ready = trn_df[target_encode_feature_list+one_hot_encode_feature_list]

In [9]:
trn_label = trn_df[['fraud_ind']]

## Over-sampling

In [10]:
from imblearn.over_sampling import RandomOverSampler

In [11]:
ros = RandomOverSampler(random_state=0)
trn_ready_resampled, trn_label_resampled = ros.fit_resample(trn_ready, trn_label)
trn_ready_resampled = pd.DataFrame(trn_ready_resampled, columns=target_encode_feature_list+one_hot_encode_feature_list).infer_objects()
trn_label_resampled = pd.DataFrame(trn_label_resampled, columns=['fraud_ind']).infer_objects()
# trn_ready_resampled.columns = [col for (col,) in trn_ready_resampled.columns]

C:\Users\tim19\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
contp = tf.feature_column.categorical_column_with_vocabulary_list('contp', [5, 3, 6, 2, 4, 0, 1])
etymd = tf.feature_column.categorical_column_with_vocabulary_list('etymd', [0, 2, 5, 4, 8, 1, 9, 6, 3, 7, 10])
iterm = tf.feature_column.categorical_column_with_vocabulary_list('iterm', [0, 1, 2, 8, 3, 4, 7, 6, 5])
stocn = tf.feature_column.categorical_column_with_vocabulary_list('stocn', list(trn_ready_resampled.stocn.unique()))
hcefg = tf.feature_column.categorical_column_with_vocabulary_list('hcefg', [5, 0, 3, 1, 2, 7, 8, 9, 6])
csmcu = tf.feature_column.categorical_column_with_vocabulary_list('csmcu', list(trn_ready_resampled.csmcu.unique()))
# Binary
ecfg = tf.feature_column.categorical_column_with_vocabulary_list('ecfg', ['N', 'Y'])
insfg =tf.feature_column.categorical_column_with_vocabulary_list('insfg', ['N', 'Y'])
flbmk = tf.feature_column.categorical_column_with_vocabulary_list('flbmk', list(trn_ready_resampled.flbmk.unique()))
flg_3dsmk = tf.feature_column.categorical_column_with_vocabulary_list('flg_3dsmk', list(trn_ready_resampled.flg_3dsmk.unique()))

contp_one_hot = tf.feature_column.indicator_column(contp)
etymd_one_hot = tf.feature_column.indicator_column(etymd)
iterm_one_hot = tf.feature_column.indicator_column(iterm)
stocn_one_hot = tf.feature_column.indicator_column(stocn)
hcefg_one_hot = tf.feature_column.indicator_column(hcefg)
csmcu_one_hot = tf.feature_column.indicator_column(csmcu)
ecfg_one_hot = tf.feature_column.indicator_column(ecfg)
insfg_one_hot = tf.feature_column.indicator_column(insfg)
flbmk_one_hot = tf.feature_column.indicator_column(flbmk)
flg_3dsmk_one_hot = tf.feature_column.indicator_column(flg_3dsmk)

feature_cols = [
    contp_one_hot, etymd_one_hot, iterm_one_hot, stocn_one_hot, hcefg_one_hot, csmcu_one_hot, ecfg_one_hot, insfg_one_hot,
    flbmk_one_hot, flg_3dsmk_one_hot
]

In [13]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import sys
import tensorflow as tf

In [14]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(trn_ready_resampled, trn_label_resampled, test_size=0.3)

In [15]:
# x_train = pd.DataFrame(data=x_train, columns=[target_encode_feature_list+one_hot_encode_feature_list])
# x_test = pd.DataFrame(data=x_test, columns=[target_encode_feature_list+one_hot_encode_feature_list])
# y_train = pd.DataFrame(data=y_train, columns=['fraud_ind'])
# y_test = pd.DataFrame(data=y_test, columns=['fraud_ind'])

In [16]:
def numpy_input_fn(features, labels=None, training=True, batch_size=256):
    
    """An input function for training or evaluating"""
    # 将输入转换为数据集。
    if labels is not None:
#         labels = labels.reshape(-1,)
#         inputs = (dict(features), labels)
        dataset = tf.data.Dataset.from_tensor_slices((features))
#         dataset = tf.convert_to_tensor((features))
    else:
#         inputs = (dict(features))
        dataset = tf.data.Dataset.from_tensor_slices((features))
#     dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    
    # 如果在训练模式下混淆并重复数据。
    if training:
        dataset = dataset.shuffle(1000).repeat()
    dataset = dataset.batch(batch_size)
    return dataset

In [17]:
def input_fn(features, labels=None, training=True, batch_size=256):
    
    """An input function for training or evaluating"""
    # 将输入转换为数据集。
    if labels is not None:
        labels = labels.values.reshape(-1,)
        inputs = (dict(features), labels)
    else:
        inputs = (dict(features))
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    # 如果在训练模式下混淆并重复数据。
    if training:
        dataset = dataset.shuffle(1000).repeat()
    dataset = dataset.batch(batch_size)
    return dataset

In [18]:
n_batches = 1
est = tf.estimator.BoostedTreesClassifier(
            feature_columns=feature_cols,
            n_batches_per_layer=n_batches,
            n_classes=2,
#             n_trees=500,
#             max_depth=5,
            learning_rate=0.001,
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\tim19\\AppData\\Local\\Temp\\tmp96bptnlb', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000195BB7317F0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for 

In [19]:
# new test
results = est.train(input_fn=lambda: input_fn(x_train, y_train), steps=100000)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably 

In [20]:
evaluations = est.evaluate(input_fn=lambda: input_fn(x_test, y_test, training=False))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-10-28T00:04:28Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\tim19\AppData\Local\Temp\tmp96bptnlb\model.ckpt-600
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-10-28-00:05:25
INFO:tensorflow:Saving dict for global step 600: accuracy = 0.90738034, accuracy_baseline = 0.50018245, auc = 0.9544427, auc_precision_recall = 0.95137113, average_loss = 0.43926996, global_step = 600, label/mean = 0.49981755, loss = 0.43927002, precision = 0.8826707, prediction/mean = 0.50105387, recall = 0.93958807
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:

In [21]:
evaluations

{'accuracy': 0.90738034,
 'accuracy_baseline': 0.50018245,
 'auc': 0.9544427,
 'auc_precision_recall': 0.95137113,
 'average_loss': 0.43926996,
 'label/mean': 0.49981755,
 'loss': 0.43927002,
 'precision': 0.8826707,
 'prediction/mean': 0.50105387,
 'recall': 0.93958807,
 'global_step': 600}

In [22]:
trn_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1509206 entries, 0 to 1518246
Data columns (total 16 columns):
locdt        1509206 non-null int64
loctm        1509206 non-null object
contp        1509206 non-null int64
etymd        1509206 non-null int64
mcc          1509206 non-null int64
conam        1509206 non-null float64
ecfg         1509206 non-null object
insfg        1509206 non-null object
iterm        1509206 non-null int64
stocn        1509206 non-null int64
stscd        1509206 non-null int64
hcefg        1509206 non-null int64
csmcu        1509206 non-null int64
flbmk        1509206 non-null object
flg_3dsmk    1509206 non-null object
fraud_ind    1509206 non-null int64
dtypes: float64(1), int64(10), object(5)
memory usage: 195.7+ MB
